In [1]:
import random
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from tqdm import tqdm
from tabulate import tabulate
import os
import pandas as pd
from PIL import Image
import imageio

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Subset
from torchvision.datasets import MNIST

## Loss Definitions

In [2]:


# Create nn.module class called ArcFace that can be plugged in at the end of any backbone network

class ArcFaceLayer(nn.Module):
    def __init__(self, in_features, num_classes, s=4, m=0.5):
        super(ArcFaceLayer, self).__init__()

        #Margin parameter and scaling factor
        self.s = s
        self.m = m

        #Input feature dimension and output number of classes
        self.in_features = in_features
        self.num_classes = num_classes

        self.weight = nn.Parameter(torch.FloatTensor(num_classes, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, x, labels):

        #Cosine similarity between normalized input features and normalized weights
        cos_theta = F.linear(F.normalize(x), F.normalize(self.weight))

        # Clamp the values between -1 and 1 with 1e-7 for numerical stability
        cos_theta = torch.clamp(cos_theta, -1 + 1e-7, 1 - 1e-7)

        #Get the angle using arccos
        theta = torch.acos(cos_theta)

        #Add margin to the angle
        theta += self.m

        #Apply cosine to the angle to get adjusted cosine similarity
        adjusted_cos_theta = torch.cos(theta)

        #One hot encode labels
        one_hot = torch.zeros(cos_theta.size(), device=x.device)

        #Fill the one hot encoded tensor with 1s at the label indices
        one_hot.scatter_(1, labels.view(-1, 1).long(), 1)

        output = (one_hot * adjusted_cos_theta) + ((1.0 - one_hot) * cos_theta)
        output *= self.s

        return output

## Model Definitions

In [3]:


# Create a model class that combines the backbone network (Resnet18) and (optinally) the ArcFace layer
# The model class will be used for training and inference

# Create a base class called EmbeddingModel that contains the backbone network
class EmbeddingModel(nn.Module):
    def __init__(self, embedding_size):
        super(EmbeddingModel, self).__init__()
        self.embedding_size = embedding_size

        # Load a pre-trained backbone network (e.g., ResNet18)
        self.backbone = models.resnet18(weights= models.ResNet18_Weights.DEFAULT)

        # Freeze the backbone network
        for param in self.backbone.parameters():
            param.requires_grad = False


        # Replace the last fc layer of the backbone network with a one that outputs embedding_size
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, self.embedding_size)


# ArcFaceModel is a subclass of EmbeddingModel that adds an ArcFace layer
class ArcFaceModel(EmbeddingModel):
    def __init__(self, num_classes, embedding_size):
        super(ArcFaceModel, self).__init__(embedding_size)

        # Number of classes
        self.num_classes = num_classes

        # Create an new ArcFace layer
        self.arcface = ArcFaceLayer(self.embedding_size, self.num_classes)

    def forward(self, x, labels=None):

        # Get the output of the backbone network, so that we can use it to compute the embedding vectors
        x = self.backbone(x)

        # A cpu copy of the embedding vectors with detached gradients
        embedding_vectors = x.cpu().detach()

        output = self.arcface(x, labels)

        return output, embedding_vectors

    # Add a method to get the embedding vector
    def get_embedding(self, x):
        #no grad
        with torch.no_grad():
            return self.backbone(x)

class VGGBlock(nn.Module):
    def __init__(self, in_channels, out_channels, num_layers):
        super(VGGBlock, self).__init__()
        layers = []
        for _ in range(num_layers):
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
            layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU(inplace=True))
            in_channels = out_channels
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)

class VGG8ArcFace(nn.Module):
    def __init__(self, num_features, num_classes):
        super(VGG8ArcFace, self).__init__()
        self.block1 = VGGBlock(1, 16, 2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.block2 = VGGBlock(16, 32, 2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.block3 = VGGBlock(32, 64, 2)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.batchnorm = nn.BatchNorm2d(64)
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 3 * 3, num_features)
        self.batchnorm2 = nn.BatchNorm1d(num_features)
        self.arcface = ArcFaceLayer(num_features, num_classes)

    # Forward pass, returns the embedding vectors and the output of the ArcFace layer
    def forward(self, x, labels):
        x = self.block1(x)
        x = self.maxpool1(x)
        x = self.block2(x)
        x = self.maxpool2(x)
        x = self.block3(x)
        x = self.maxpool3(x)
        x = self.batchnorm(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.batchnorm2(x)
        embedding_vectors = x.cpu().detach()
        output = self.arcface(x, labels)

        return output, embedding_vectors

    def get_embedding(self, x):

        #no grad
        with torch.no_grad():
            x = self.block1(x)
            x = self.maxpool1(x)
            x = self.block2(x)
            x = self.maxpool2(x)
            x = self.block3(x)
            x = self.maxpool3(x)
            x = self.batchnorm(x)
            x = self.dropout(x)
            x = self.flatten(x)
            x = self.fc1(x)
            x = self.batchnorm2(x)
            return x



## Visualization Functions

In [4]:
def visualize_embeddings(all_embeddings, all_labels, visualize_val=False):

    # First plot train embeddings
    plot_embeddings(all_embeddings['train'], all_labels['train'], title_poststr="Training_Embeddings")

    if visualize_val:
        # Then plot val embeddings
        plot_embeddings(all_embeddings['val'], all_labels['val'], title_poststr="Validation_Embeddings")


# Function to plot the embeddings using matplotlib
def plot_embeddings(embeddings, labels, title_poststr = "Training_Embeddings"):

    #Get the number of epochs
    num_epochs = len(embeddings)

    #Num classes
    num_classes = len(np.unique(labels[0]))

    # Create a GIF of the embeddings

    #Create list to hold frames for train and val
    frames = []

    #Get the min and max values for the train embeddings across all epochs
    x_min = np.min([np.min(embeddings[epoch][:,0]) for epoch in range(num_epochs)])
    x_max = np.max([np.max(embeddings[epoch][:,0]) for epoch in range(num_epochs)])
    y_min = np.min([np.min(embeddings[epoch][:,1]) for epoch in range(num_epochs)])
    y_max = np.max([np.max(embeddings[epoch][:,1]) for epoch in range(num_epochs)])

    #max absolute value
    max_abs = max(abs(x_min), abs(x_max), abs(y_min), abs(y_max))

    # Set the limits of the axes
    #xlim = [x_min - 0.1, x_max + 0.1]
    #ylim = [y_min - 0.1, y_max + 0.1]
    xlim = [-max_abs - 0.1, max_abs + 0.1]
    ylim = [-max_abs - 0.1, max_abs + 0.1]

    #Create gif for train
    for epoch in range(num_epochs):
        epoch_embeddings = embeddings[epoch]
        epoch_labels = labels[epoch]

        #Create a scatter plot with colored labels
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(111)
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)

        # Define a custom colormap excluding the color red
        colors = ['blue', 'green', 'yellow', 'cyan', 'magenta', 'orange', 'purple']
        cmap = ListedColormap(colors)

        # Add a unit circle to the plot
        circle = plt.Circle((0,0), 1, color='red', fill=False)
        ax.add_artist(circle)

        for class_id in range(num_classes):
            class_indices = np.where(epoch_labels == class_id)[0]
            ax.scatter(epoch_embeddings[class_indices, 0], epoch_embeddings[class_indices, 1], label=f"Class {class_id}", alpha=1)



            # Add axes to the plot with arrows
            ax.arrow(xlim[0], 0, xlim[1] - xlim[0], 0, length_includes_head=True, head_width=0.05, color='black')
            ax.arrow(0, ylim[0], 0, ylim[1] - ylim[0], length_includes_head=True, head_width=0.05, color='black')

        plt.title(f"Epoch [{epoch+1}/{num_epochs}] - {title_poststr}")
        plt.xlabel('Dimension 1')
        plt.ylabel('Dimension 2')
        #Set the limits of the axes
        fig.gca().set_xlim([xlim[0]-0.1, xlim[1]+0.1])
        fig.gca().set_ylim([ylim[0]-0.1, ylim[1]+0.1])

        # Add the unit circle to the legend
        ax.scatter([], [], color='red', label="Unit Circle")

        plt.legend()

        #Save the plot as an image
        plt.savefig(f"data/{title_poststr}_{epoch}.png")
        plt.close(fig)

        #Add the image to the list of frames
        frames.append(imageio.imread(f"data/{title_poststr}_{epoch}.png"))

    #Save the list of frames as a GIF
    imageio.mimsave(f"data/{title_poststr}.gif", frames, duration=125)


## Hyperparameters

In [5]:
# Hyperparameters
num_epochs = 20
batch_size = 4
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the random seed for reproducible results
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

## Training Loop

In [6]:
# Function to train the model
def train(model, train_loader, val_loader, optimizer, criterion, num_epochs, save_embeddings=False, visualize_val=False):
    best_val_loss = float("inf")  # Initialize with a very high value
    model.train()

    train_loop = tqdm(total=len(train_loader), leave=False)
    train_loop.set_description(f"Epochs: 0/{num_epochs}")

    table_data = []  # Table data to store epoch, train loss, and val loss

    #Dictionary to store the embeddings and labels for visualization
    all_embeddings = {}
    all_labels = {}

    all_embeddings['train'] = []
    all_embeddings['val'] = []

    all_labels['train'] = []
    all_labels['val'] = []

    for epoch in range(num_epochs):
        total_loss = 0.0

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output, train_batch_embeddings = model(data, target)  # Pass both data and target to the model's forward method

            # if epoch == 2:
            #     print(train_batch_embeddings)

            #Save the embeddings and labels for visualization
            if save_embeddings:
                if batch_idx == 0:
                    train_embeddings = train_batch_embeddings
                    train_labels = target.cpu().detach()
                else:
                    train_embeddings = torch.cat((train_embeddings, train_batch_embeddings), dim=0)
                    train_labels = torch.cat((train_labels, target.cpu().detach()), dim=0)

            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            train_loop.update(1)

        train_loss = total_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for batch_idx, (val_data, val_target) in enumerate(tqdm(val_loader, leave=False)):
                val_data, val_target = val_data.to(device), val_target.to(device)
                val_output, val_batch_embeddings = model(val_data, val_target)  # Pass both val_data and val_target to the model's forward method

                if save_embeddings:
                    if batch_idx == 0:
                        val_embeddings = val_batch_embeddings
                        val_labels = val_target.cpu().detach()
                    else:
                        val_embeddings = torch.cat((val_embeddings, val_batch_embeddings), dim=0)
                        val_labels = torch.cat((val_labels, val_target.cpu().detach()), dim=0)

                val_loss += criterion(val_output, val_target).item()

        val_loss /= len(val_loader)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "models/best_model.pth")  # Save the model with the best validation loss

        #Save the embeddings and labels for visualization
        if save_embeddings:
            all_embeddings['train'].append(train_embeddings.numpy())
            all_embeddings['val'].append(val_embeddings.numpy())

            all_labels['train'].append(train_labels.numpy())
            all_labels['val'].append(val_labels.numpy())

        train_loop.set_description(f"Epochs: {epoch+1}/{num_epochs}")
        train_loop.set_postfix(train_loss=train_loss, val_loss=val_loss)

        train_loop.reset()

        # Store epoch, train loss, and val loss in table data
        table_data.append([epoch+1, train_loss, val_loss])

        # Create a table with epoch, train loss, and val loss
        table = tabulate(table_data, headers=["Epoch", "Train Loss", "Val Loss"], tablefmt="presto")

        # Print the table with updated data
        if epoch == 0:
            # First epoch, print the table normally
            print(table)
        else:
            # Subsequent epochs, use carriage return to overwrite the previous table
            table = tabulate(table_data, headers=["Epoch", "Train Loss", "Val Loss"], tablefmt="presto")
            print(table, end="\r")

    train_loop.close()

    print("Training complete!")

    #Create a visualization of the embeddings
    if save_embeddings:
        visualize_embeddings(all_embeddings, all_labels, visualize_val= visualize_val)




In [7]:

# Transforms to be applied on the MNIST images
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

#Load MNIST dataset
dataset = MNIST(root="data/", download=True, transform=transform)

#number of classes
num_classes = len(dataset.classes)


# Split the dataset into training, validation and test sets

# Define the split sizes
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Use the random_split function to split dataset into non-overlapping training, validation and test sets
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])



# Create data loaders with transforms, no shuffling for repeatable results
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Create an instance of ArcFaceModel
#model = ArcFaceModel(num_classes=num_classes, embedding_size=2).to(device)
model = VGG8ArcFace(num_classes=num_classes, num_features=2).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model using train() function
train(model, train_loader, val_loader, optimizer, criterion, num_epochs, save_embeddings=True, visualize_val=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 145459641.96it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 42563490.45it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 35945007.83it/s]


...

Epochs: 19/20:   0%|          | 16/12000 [00:00<01:20, 149.77it/s, train_loss=1.05, val_loss=1.06]

   Epoch |   Train Loss |   Val Loss
---------+--------------+------------
       1 |      2.91419 |    1.8126
       2 |      1.34318 |    1.18946
       3 |      1.19823 |    1.14243
       4 |      1.14477 |    1.12383
       5 |      1.11325 |    1.13186
       6 |      1.10478 |    1.09975
       7 |      1.08924 |    1.09742
       8 |      1.08683 |    1.09093
       9 |      1.07598 |    1.09364
      10 |      1.07207 |    1.08653
      11 |      1.06442 |    1.07252
      12 |      1.05747 |    1.07775
      13 |      1.0615  |    1.08621
      14 |      1.05388 |    1.08023
      15 |      1.05785 |    1.06531
      16 |      1.05592 |    1.08674
      17 |      1.05188 |    1.06806
      18 |      1.051   |    1.09049


 98%|█████████▊| 1472/1500 [00:04<00:00, 309.07it/s]


   Epoch |   Train Loss |   Val Loss
---------+--------------+------------
       1 |      2.91419 |    1.8126
       2 |      1.34318 |    1.18946
       3 |      1.19823 |    1.14243
       4 |      1.14477 |    1.12383
       5 |      1.11325 |    1.13186
       6 |      1.10478 |    1.09975
       7 |      1.08924 |    1.09742
       8 |      1.08683 |    1.09093
       9 |      1.07598 |    1.09364
      10 |      1.07207 |    1.08653
      11 |      1.06442 |    1.07252
      12 |      1.05747 |    1.07775
      13 |      1.0615  |    1.08621
      14 |      1.05388 |    1.08023
      15 |      1.05785 |    1.06531
      16 |      1.05592 |    1.08674
      17 |      1.05188 |    1.06806
      18 |      1.051   |    1.09049
      19 |      1.04746 |    1.06095
Training complete!


<ipython-input-4-9034b681a918>:86: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(f"data/{title_poststr}_{epoch}.png"))
